In [3]:
import duckdb
import pandas as pd

In [4]:
con = duckdb.connect('../data/diamonds_train.db')

In [32]:
df = con.execute('show tables').fetch_df()
df

,name
0,diamonds_city
1,diamonds_clarity
2,diamonds_color
3,diamonds_cut
4,diamonds_dimensions
5,diamonds_properties
6,diamonds_transactional
7,sqlite_autoindex_diamonds_city_1
8,sqlite_autoindex_diamonds_clarity_1
9,sqlite_autoindex_diamonds_color_1


In [14]:
df1 = []
for i, row in df.iterrows():
    df1.append(con.execute('SELECT * FROM ' + row[0]).fetch_df())
    if (i > 5):
        break

In [54]:
df1[4]

,index_id,depth,table,x,y,z
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95
...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,62.7,57.0,7.10,7.04,4.43
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,57.1,60.0,8.31,8.25,4.73
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,62.7,56.0,6.37,6.42,4.01
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,61.9,54.3,4.45,4.47,2.76


In [55]:
df_diamonds = con.execute(
'''SELECT dt.index_id, dc.city, dt.price, dt.carat, dd.x, dd.y, dd.z, dd.table, dd.depth, dcl.clarity, dcolor.color, dcut.cut 
FROM diamonds_transactional AS dt
INNER JOIN diamonds_city AS dc ON dt.city_id = dc.city_id
INNER JOIN diamonds_properties AS dp ON dt.index_id = dp.index_id
INNER JOIN diamonds_clarity AS dcl ON dp.clarity_id = dcl.clarity_id
INNER JOIN diamonds_color AS dcolor ON dp.color_id = dcolor.color_id
INNER JOIN diamonds_cut AS dcut ON dp.cut_id = dcut.cut_id
INNER JOIN diamonds_dimensions AS dd ON dt.index_id = dd.index_id
''').fetch_df()

In [56]:
df_diamonds

,index_id,city,price,carat,x,y,z,table,depth,clarity,color,cut
0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Kimberly,505,0.32,4.35,4.38,2.75,57.0,63.0,VS2,H,Very Good
1,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,Las Vegas,2686,0.71,5.62,5.53,3.65,55.0,65.5,VS1,G,Fair
2,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,Kimberly,738,0.41,4.68,4.72,3.00,56.0,63.8,SI1,D,Good
3,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,Tel Aviv,9057,1.52,7.45,7.39,4.54,57.0,61.2,SI2,F,Ideal
4,2c624232cdd221771294dfbb310aca000a0df6ac8b66b6...,Zurich,557,0.31,4.33,4.37,2.69,54.2,61.8,VS1,H,Ideal
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,19753adcd63b8c9ecefd5b4587cc24d9e736bbca3b6da7...,Amsterdam,1116,0.35,4.52,4.45,2.79,58.0,62.2,IF,G,Premium
40451,ce1f611041ac7d8d494958c79e4794c1630d3fef158572...,Luxembourg,975,0.40,4.74,4.71,2.95,55.0,62.4,VS2,E,Ideal
40452,08ea07799da9b8948f1df2e339def3db7af84e92ac2aa5...,Tel Aviv,8572,1.57,7.46,7.43,4.55,61.0,61.1,SI2,G,Premium
40453,ec7cec1932d9322dc6b60734d98cf2fc12cf68abbf2718...,Paris,5154,0.73,5.80,5.81,3.56,56.0,61.4,VVS1,F,Ideal


In [73]:
df_diamonds.mean(numeric_only=True)

price    3928.444469
carat       0.797706
x           5.729392
y           5.732819
z           3.537154
table      57.446133
depth      61.752841
dtype: float64

In [78]:
df_diamonds.median(numeric_only=True)

price    2397.00
carat       0.70
x           5.69
y           5.71
z           3.52
table      57.00
depth      61.80
dtype: float64

In [74]:
df_diamonds.max(numeric_only=True)

price    18823.00
carat        4.50
x           10.23
y           58.90
z            8.06
table       95.00
depth       79.00
dtype: float64

In [75]:
df_diamonds.min(numeric_only=True)

price    326.0
carat      0.2
x          0.0
y          0.0
z          0.0
table     43.0
depth     43.0
dtype: float64

In [77]:
df_diamonds.mode()

,index_id,city,price,carat,x,y,z,table,depth,clarity,color,cut
0,0002a1cf14e9c1acaa8255fd6777d916d3aec6bc1f3c8a...,Surat,605.0,0.3,4.37,4.37,2.69,56.0,61.9,SI1,G,Ideal
1,0002dc999ce736640267221e6a004da4b837667ca7de5c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00037f39cf870a1f49129f9c82d935665d352ffd25ea32...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00039a15178b11924de22fd1a02f6efb00d8af33c171a6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0007256972781977cbceab0e2cbdc57e20512a6f297529...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,fff81139a7dd8a3a48f462347a5611b8e0b84258237d5d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40451,fffb2bcdee8b153cbd63617ddc29b4ecf529e15a20ed3c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40452,fffb7503311e0a0216918886db1208459195da3102267e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40453,fffeb5d21b5a1160a3fe9a4de54851b3f2f96e3d6c7e7c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df_diamonds.groupby('color').sum(numeric_only=True).sort_values('carat', ascending = False)

,price,carat,x,y,z,table,depth
color,,,,,,,
G,34016282,6547.40,48059.08,48085.13,29667.38,484369.1,522119.8
H,28098796,5714.30,37545.44,37562.08,23184.91,361137.8,388091.1
F,26392382,5250.00,40201.20,40243.26,24803.21,412029.6,442748.8
E,22622109,4825.97,39656.77,39691.62,24456.04,421147.7,451658.3
I,20526383,4140.02,25063.31,25061.60,15491.26,232092.3,249472.8
D,15828328,3297.18,27288.35,27312.77,16847.41,289534.7,311633.8
J,11440941,2496.31,13968.39,13964.72,8645.34,123672.1,132486.6


array(['Kimberly', 'Las Vegas', 'Tel Aviv', 'Zurich', 'Antwerp', 'Madrid',
       'Paris', 'Surat', 'Amsterdam', 'Luxembourg', 'London',
       'New York City', 'Dubai'], dtype=object)